# Machine Learning Final Project - Classification of Threat and Safety in Brain Imaging

## Description

I've been involved in some previous work in looking at fMRI data in classification tasks.  In this project I'm interested in exploring this technique more and hopefully learning something interesting about brain imaging and how we can use classification to understand it better.

This project will take brain images from a study that was performed looking at individuals responses to a sound, then an immediate electric shock, then just the sound in one instance.  And in the other instance a sound, *no* electric shock, the same sound again.

The idea here is that the first set of individuals will fear the second sound they here.  And the second set of individuals will experience a lack of fear.

If this is the experience felt by the individuals, we can then classify which regions in the brain (from the fMRI scan) are areas associated with a fear response.

## Exploratory Data Analysis

In [17]:
import glob
import numpy as np
import os #for paths
import nibabel as nib
np.set_printoptions(precision=2, suppress=True) #2decimals
from nilearn.input_data import NiftiMasker
import matplotlib.pyplot as plt

def show_slices(slices):
 """ Display row of image slices """
 fig, axes = plt.subplots(1, len(slices))
 for i, slice in enumerate(slices):
     axes[i].imshow(slice.T, cmap="gray", origin="lower")

acq_Z_sig=nib.load('/storage/workspace/google_drive/pabo6156@colorado.edu/ML_Final_Proj/acq_Z_wtmap.nii')
acq_data=acq_Z_sig.get_fdata()
print(acq_data)
#acq_affine=acq_Z_sig.get_affine()
#print(acq_affine)
#acq_header=acq_Z_sig.get_header()
#print(acq_header)

# con 1 = csp > baseline, con2 csm >baseline
cs_data=sorted(glob.glob('/storage/workspace/google_drive/pabo6156@colorado.edu/ML_Final_Proj/fmri/model/con_000[1-2].img'))
print(cs_data)

# load behavioral labels
filename='/storage/workspace/google_drive/pabo6156@colorado.edu/ML_Final_Proj/fmri/acq_labels.csv'
labels=np.recfromcsv(filename) #this will skip first col
print(labels)

# from nilearn import datasets
# haxby_dataset = datasets.fetch_haxby()
mask_filename = '/storage/workspace/google_drive/pabo6156@colorado.edu/ML_Final_Proj/fmri/BinaryWholeBrain.nii'
nifti_masker = NiftiMasker(mask_img=mask_filename, standardize=False)
fmri_masked = nifti_masker.fit_transform(cs_data)


# cs p
img_csp=nib.load('/storage/workspace/google_drive/pabo6156@colorado.edu/ML_Final_Proj/fmri/model1/con_0001.img')
header=img_csp.get_header()
print(header)
epi_img_data = img_csp.get_data()


slice_0 = epi_img_data[26, :, :]
slice_1 = epi_img_data[:, 30, :]
slice_2 = epi_img_data[:, :, 16]
show_slices([slice_0, slice_1, slice_2])
plt.suptitle("Center slices for EPI image")  


# Verify
data = img.get_data() # raw scans bundled in a numpy array
affine = img.get_affine() # voxel index & spatial location
header = img.get_header() # info about data collection

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[]
[(b'csp

/Users/ebcdic/.pyenv/versions/3.11.1/lib/python3.11/site-packages/numpy/lib/npyio.py:2538: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


TypeError: Cannot concatenate empty objects

## Train Radial Basis Function SVM Classifier

In [45]:
from sklearn.svm import SVC
from sklearn.cross_validation import LeaveOneLabelOut

svc = SVC(kernel='rbf')
svc.fit(fmri_masked, labels)
prediction = svc.predict(fmri_masked)



SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [46]:
from sklearn.cross_validation import KFold
target=labels
cv = KFold(n=len(fmri_masked), n_folds=5)
cv_scores = []

for train, test in cv:
    svc.fit(fmri_masked[train], target[train])
    prediction = svc.predict(fmri_masked[test])
    cv_scores.append(np.sum(prediction == target[test])
                     / float(np.size(target[test])))

print(cv_scores)

[0.5714285714285714, 0.66666666666666663, 0.85185185185185186, 0.7407407407407407, 0.70370370370370372]


In [47]:
from sklearn.cross_validation import cross_val_score
cv_scores = cross_val_score(svc, fmri_masked, target, cv=cv)
print(cv_scores) 
classification_accuracy = np.mean(cv_scores)
print(classification_accuracy)

[ 0.57  0.67  0.85  0.74  0.7 ]
0.706878306878


In [36]:
# cv = LeaveOneLabelOut(labels=target) 
# cv_scores = cross_val_score(svc, fmri_masked, target, cv=cv)
# print(cv_scores)
# classification_accuracy = np.mean(cv_scores)

## Train and test the null hypothesis

In [48]:
from sklearn.dummy import DummyClassifier
null_cv_scores = cross_val_score(DummyClassifier(), fmri_masked, target, cv=cv)
print(null_cv_scores)

from sklearn.cross_validation import permutation_test_score
null_cv_scores = permutation_test_score(svc, fmri_masked, target, cv=cv)
print(null_cv_scores)

[ 0.46  0.33  0.59  0.63  0.44]


## Show the weights of the SVM

In [49]:
# Retrieve the SVC discriminating weights
coef_ = svc.coef_

# Reverse masking thanks to the Nifti Masker
coef_img = nifti_masker.inverse_transform(coef_)

# Save the coefficients as a Nifti image
coef_img.to_filename('acq_linear_svc_weights.nii')

### Visualization #############################################################
import matplotlib.pyplot as plt
from nilearn.image.image import mean_img
from nilearn.plotting import plot_roi, plot_stat_map

mean_epi = mean_img(cs_data)
plot_stat_map(coef_img, mean_epi, title="SVM weights", display_mode="yx")

plot_roi(nifti_masker.mask_img_, mean_epi, title="Mask", display_mode="yx")

plt.show()